In [75]:
import os
import json
import pandas as pd
import traceback

In [76]:
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
mykey=os.getenv("OPENAI_API_KEY")

In [78]:

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key=mykey,
    model="gpt-4o-mini",
    temperature=0.4 
)

In [79]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [100]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Response json must be a valid json . Do not send invalid jsons .  Return ONLY a valid JSON object matching this schema (no markdown, no comments, no prose):\n
        "{response_json}\n"\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [101]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [102]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}


In [103]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [104]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.\
If the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilities.\
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [105]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subjects","quiz"],template=TEMPLATE2)

In [106]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [108]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)


In [109]:
file_path="/Users/aryannayar/Downloads/mcqgen/data.txt"

In [110]:
with open(file_path,"r") as file:
    TEXT=file.read()

In [134]:
NUMBER=10,
SUBJECT="Cat study",
TONE="VERY HARD"

In [135]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [136]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain({
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The cat (Felis catus), also referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and working cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator.

Cat intelligence is evident in their ability to adapt, learn through observation, and solve problems. Research has shown they possess strong memories, exhibit neuroplasticity, and display cognitive sk

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for ('Cat study',) students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.If the quiz is not at par with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilities.Quiz_MCQs:
{"1": {"mcq": "What is the scientific name for the domestic cat?", "options": {"a": "Felis lybica", "b": "Felis catus", "c": "Felis silvestris", "d": "Felis daemon"}, "correct": "b"}, "2": {"mcq": "Which gene is linked to coat color variation in domestic cats?", "options": {"a": "MC1R", "b": "ASIP", "c": "Felis", "d": "Proto-Uralic"}, "correct": "b"}, "3": {"mcq": "At what age did the domestication of cats begin according to archaeological findings?", "options": {"a": "3100 BC", "b": "7500 

In [137]:
print("\n--- Usage Stats ---")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost:.6f}")



--- Usage Stats ---
Total Tokens: 6459
Prompt Tokens: 5503
Completion Tokens: 956
Total Cost: $0.001399


In [138]:
response

{'text': 'The cat (Felis catus), also referred to as the domestic cat or house cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and working cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator.\n\nCat intelligence is evident in their ability to adapt, learn through observation, and solve problems. Research has shown they possess strong memories, exhibit neuroplasticity, and display cognitive skills comparable to 

In [139]:
quiz=response.get("quiz")

In [140]:
quiz=json.loads(quiz)

In [141]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=" | ".join(
        [
            f"{option}: {option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct= value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [142]:
quiz_table_data

[{'MCQ': 'What is the scientific name for the domestic cat?',
  'Choices': 'a: Felis lybica | b: Felis catus | c: Felis silvestris | d: Felis daemon',
  'Correct': 'b'},
 {'MCQ': 'Which gene is linked to coat color variation in domestic cats?',
  'Choices': 'a: MC1R | b: ASIP | c: Felis | d: Proto-Uralic',
  'Correct': 'b'},
 {'MCQ': 'At what age did the domestication of cats begin according to archaeological findings?',
  'Choices': 'a: 3100 BC | b: 7500 BC | c: 5500 BC | d: 1200 BC',
  'Correct': 'b'},
 {'MCQ': 'What term describes a group of cats?',
  'Choices': 'a: Pack | b: Clowder | c: Colony | d: Glaring',
  'Correct': 'b'},
 {'MCQ': 'What is the average weight range of an adult domestic cat?',
  'Choices': 'a: 2-4 kg | b: 4-5 kg | c: 5-7 kg | d: 7-10 kg',
  'Correct': 'b'},
 {'MCQ': 'Which of the following senses is most acute in domestic cats?',
  'Choices': 'a: Taste | b: Vision | c: Hearing | d: Touch',
  'Correct': 'c'},
 {'MCQ': 'What reflex allows cats to right themselves

In [143]:
quiz=pd.DataFrame(quiz_table_data)

In [144]:
quiz.to_csv("machinelearning.csv",index=False)